<a href="https://colab.research.google.com/github/Dr-Isam-ALJAWARNEH/fds-project-airnav/blob/main/Cleaning_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Import Google Colab file upload functionality
from google.colab import files
uploaded = files.upload()  # Use the file chooser to upload all CSV parts

Saving chicago_eclipse_data_part_1.csv to chicago_eclipse_data_part_1.csv
Saving chicago_eclipse_data_part_2.csv to chicago_eclipse_data_part_2.csv
Saving chicago_eclipse_data_part_3.csv to chicago_eclipse_data_part_3.csv


In [3]:
# Import the datascience Table abstraction
from datascience import Table

In [4]:
# Get the list of uploaded filenames and sort them (if needed)
file_names = sorted(uploaded.keys())
print("Uploaded files:", file_names)

Uploaded files: ['chicago_eclipse_data_part_1.csv', 'chicago_eclipse_data_part_2.csv', 'chicago_eclipse_data_part_3.csv']


In [16]:
# Read each CSV file into a Table and collect them in a list
tables = []
for file in file_names:
    try:
        t = Table.read_table(file)
        tables.append(t)
        print(f"Loaded {file} with {t.num_rows} rows.")
    except Exception as e:
        print(f"Error reading {file}: {e}")

Loaded chicago_eclipse_data_part_1.csv with 129531 rows.
Loaded chicago_eclipse_data_part_2.csv with 129531 rows.
Loaded chicago_eclipse_data_part_3.csv with 129531 rows.


In [20]:
# Use the first table as the master with the desired column order.
master_table = tables[0]
common_labels = master_table.labels

def align_rows(table, common_labels):
    aligned_rows = []
    for row in table.rows:
        new_row = []
        for label in common_labels:
            if label in table.labels:
                new_row.append(row[table.labels.index(label)])
            else:
                new_row.append("")
        aligned_rows.append(new_row)
    return aligned_rows

# Convert the master table's rows to a list
all_rows = list(master_table.rows)

# Loop through each subsequent table, align its rows, and extend all_rows.
for t in tables[1:]:
    aligned = align_rows(t, common_labels)
    all_rows.extend(aligned)

# Create a new Table with the common labels and all collected rows.
merged_table = Table(common_labels, all_rows)
print("Merged table has", merged_table.num_rows, "rows and", len(merged_table.labels), "columns.")


Merged table has 0 rows and 16 columns.


In [24]:
# --- Cleaning Steps ---

# 1. Standardize column names: strip whitespace and convert to lowercase
clean_labels = [label.strip().lower() for label in merged_table.labels]

# Loop through each pair of old and new labels and update the table
for old, new in zip(merged_table.labels, clean_labels):
    merged_table = merged_table.relabeled(old, new)

print("Column names standardized.")

Column names standardized.


In [26]:
# Remove duplicate rows manually
unique_rows = []
seen = set()
for row in merged_table.rows:
    row_tuple = tuple(row)  # Convert row to tuple so it's hashable
    if row_tuple not in seen:
        seen.add(row_tuple)
        unique_rows.append(row)

# Create a new Table with only unique rows
merged_table = Table(merged_table.labels, unique_rows)
print("Removed duplicate rows. New row count:", merged_table.num_rows)


Removed duplicate rows. New row count: 0


In [27]:
# Remove rows where all entries are missing (consider missing as empty strings or None)
def row_is_empty(row):
    return all(x == '' or x is None for x in row)

clean_rows = [row for row in merged_table.rows if not row_is_empty(row)]
merged_table = Table(merged_table.labels, clean_rows)
print("Removed rows with all missing entries. New row count:", merged_table.num_rows)

Removed rows with all missing entries. New row count: 0


In [28]:
# For each string column, strip leading and trailing whitespace from each cell
cleaned_rows = []
for row in merged_table.rows:
    new_row = [x.strip() if isinstance(x, str) else x for x in row]
    cleaned_rows.append(new_row)
merged_table = Table(merged_table.labels, cleaned_rows)
print("Trimmed whitespace from string columns.")

Trimmed whitespace from string columns.


In [30]:
# Save the cleaned merged dataset to a CSV file
output_file = 'chicago_eclipse_data_cleaned.csv'
merged_table.to_csv(output_file)
print(f"Cleaned merged dataset saved as {output_file}")

Cleaned merged dataset saved as chicago_eclipse_data_cleaned.csv
